# Wetlab transfer
Author: Famke Baeuerle

This notebook serves as collection of changes to the models that came from wetlab data and expertise.

In [1]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

## Missing exchanges

So far none of the models sodium uptake reactions. This makes no sense biologically and thus we will add these components to the models.

Nickel, 34dhbz and biotin are also missing. We want to add exchanges for all of those and add 34dhbz as extracellular metabolite if its missing completely.

### 1. Sodium

Sodium is available as extracelullar metabolite in all models but not exchange reaction is given.

In [2]:
modelpaths = [
        '../../models/Cstr_14.xml', 
        '../../models/Cstr_15.xml',
        '../../models/Cstr_16.xml',
        '../../models/Cstr_17.xml',
        '../../models/Cstr_KC-Na-01.xml',
        ]

all_models = [rg.io.load_model_cobra(path) for path in modelpaths]

Look at reactions that use extracellular sodium:

In [8]:
for rea in all_models[4].metabolites.get_by_id('na1_e').reactions:
    print(rea)

PROt4: na1_e + pro__L_e --> na1_c + pro__L_c
NAt3_2: 2.0 h_e + na1_c --> 2.0 h_c + na1_e
NAabcO: atp_c + h2o_c + na1_c --> adp_c + h_c + na1_e + pi_c
GLUt4: glu__L_e + na1_e --> glu__L_c + na1_c
NAt3_15: 3.0 h_e + 2.0 na1_c --> 3.0 h_c + 2.0 na1_e
MALt5: mal__L_e + na1_e --> mal__L_p + na1_p
EX_na1_e: na1_e <=> 
ALAt4: ala__L_e + na1_e --> ala__L_c + na1_c
NAt3_1: h_e + na1_c --> h_c + na1_e


In [13]:
for model in all_models:
    try:
        model.add_boundary(model.metabolites.get_by_id('na1_e'), type="exchange")
    except (ValueError):
        print(f'EX_na1_e already exists in {model.id}')

EX_na1_e already exists in fda_1054
EX_na1_e already exists in fda_1197
EX_na1_e already exists in fda_1115
EX_na1_e already exists in fda_1116
EX_na1_e already exists in KC_Na_01


In [12]:
for model, path in zip(all_models, modelpaths):
    cobra.io.write_sbml_model(model, path)
    model_annot = rg.io.load_model_libsbml(path)
    rea = model_annot.getReaction('R_EX_na1_e')
    rg.cvterms.add_cv_term_reactions('EX_na1_e', 'BIGG', rea)
    rg.io.write_to_file(model_annot, path)

Modified model written to ../../models/Cstr_14.xml
Modified model written to ../../models/Cstr_15.xml
Modified model written to ../../models/Cstr_16.xml
Modified model written to ../../models/Cstr_17.xml
Modified model written to ../../models/Cstr_KC-Na-01.xml
